In [ ]:
import torch

In [ ]:
#Check for GPU
gpu = torch.cuda.is_available()
device = 'cuda' if gpu == True else 'cpu'
print('Using device:', device)

Using device: cpu


In [ ]:
#Connecting Google Colab with Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Directories path
train_dir = "/content/drive/MyDrive/DWT/Train"
test_dir = "/content/drive/MyDrive/DWT/Test"

In [ ]:
#Dataset image Transformation
import torchvision
from torchvision import transforms, datasets

image_size = 256
resize_transform = transforms.Compose([transforms.Resize((image_size,image_size)), 
                                       transforms.ToTensor()])

training_set = datasets.ImageFolder(root = train_dir, transform = resize_transform)
testing_set = datasets.ImageFolder(root = test_dir, transform = resize_transform)


In [ ]:
#Data Loading
from torch.utils.data import DataLoader

train_loader = DataLoader(training_set, batch_size=32, shuffle=True)
test_loader = DataLoader(testing_set, batch_size=32, shuffle=True)


In [ ]:
dataiter = iter(train_loader)
images, labels = dataiter.next()

print(images.shape)
print(labels.shape)
print(labels)
print('Training set size: ', len(training_set))
print('Test set size: ', len(testing_set))

num_epochs = 50
num_classes = 4
batch_size = 32
learning_rate = 0.0001

torch.Size([32, 3, 256, 256])
torch.Size([32])
tensor([1, 0, 2, 2, 1, 0, 3, 1, 2, 3, 0, 2, 3, 1, 2, 0, 3, 2, 1, 2, 2, 2, 0, 2,
        0, 2, 3, 0, 1, 1, 3, 2])
Training set size:  642
Test set size:  160


In [ ]:
import torch.nn as nn

class Net(nn.Module):   
  def __init__(self):
      super(Net, self).__init__()

      self.layer1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
      self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
      self.drop_out = nn.Dropout()
      self.fc1 = nn.Linear(64 * 64 * 64, 1000). to (device)
      self.fc2 = nn.Linear(1000, 4).to (device)

      

  def forward(self, x):
      out = self.layer1(x)
      out = self.layer2(out)
      out = out.reshape(out.size(0), -1)
      out = self.drop_out(out)
      out = self.fc1(out)
      out = self.fc2(out)
      return out

In [ ]:
net = Net()
if torch.cuda.is_available():
  net = net.cuda()
net

Net(
  (layer1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (drop_out): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=262144, out_features=1000, bias=True)
  (fc2): Linear(in_features=1000, out_features=4, bias=True)
)

In [ ]:
import torch.optim as optim

# defining the optimizer
optimizer = optim.Adam(net.parameters(), lr=0.0001)
# defining the loss function
criterion = nn.CrossEntropyLoss()
if torch.cuda.is_available():
    criterion = criterion.to(device)
    
print(net)

Net(
  (layer1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (drop_out): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=262144, out_features=1000, bias=True)
  (fc2): Linear(in_features=1000, out_features=4, bias=True)
)


In [ ]:
total_step = len(train_loader)
loss_list = []
acc_list = []
for epoch in range(50):
    for i, (images, labels) in enumerate(train_loader):
        # Run the forward pass
        if torch.cuda.is_available():
          images=images.cuda()
          labels=labels.cuda()
        outputs = net(images)
        if torch.cuda.is_available():
          outputs = outputs.cuda()
        loss = criterion(outputs, labels)
        loss_list.append(loss.item())

        # Backprop and perform Adam optimisation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Track the accuracy
        total = labels.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels).sum().item()
        acc_list.append(correct / total)
        print("Training")
        #if (i + 1) % 100 == 0:
        print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'.format(epoch + 1, num_epochs, i + 1, total_step, loss.item(),(correct / total) * 100))


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Training
Epoch [1/50], Step [1/21], Loss: 1.3941, Accuracy: 21.88%
Training
Epoch [1/50], Step [2/21], Loss: 2.2327, Accuracy: 21.88%
Training
Epoch [1/50], Step [3/21], Loss: 4.3755, Accuracy: 28.12%
Training
Epoch [1/50], Step [4/21], Loss: 6.9541, Accuracy: 21.88%
Training
Epoch [1/50], Step [5/21], Loss: 4.2328, Accuracy: 21.88%
Training
Epoch [1/50], Step [6/21], Loss: 2.6757, Accuracy: 43.75%
Training
Epoch [1/50], Step [7/21], Loss: 1.8074, Accuracy: 37.50%
Training
Epoch [1/50], Step [8/21], Loss: 2.8200, Accuracy: 25.00%
Training
Epoch [1/50], Step [9/21], Loss: 2.9215, Accuracy: 18.75%
Training
Epoch [1/50], Step [10/21], Loss: 1.5304, Accuracy: 53.12%
Training
Epoch [1/50], Step [11/21], Loss: 1.9730, Accuracy: 18.75%
Training
Epoch [1/50], Step [12/21], Loss: 1.4163, Accuracy: 43.75%
Training
Epoch [1/50], Step [13/21], Loss: 1.3392, Accuracy: 46.88%
Training
Epoch [1/50], Step [14/21], Loss: 1.4240, Accuracy: 40.62%
Training
Epoch [1/50], Step [15/21], Loss: 1.4098, Accura

In [2]:
net.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images=images.to (device)
        labels=labels.to (device)
        outputs = net(images).to(device)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy#: {} '.format((correct / total) * 100))


Test Accuracy#: 86.25
